# CPU Virtualization

The job of a CPU is to run the user programs.

We want to run many programs as once, N (# programs) >> M (# CPUs).

The OS makes it look like every program gets its own CPU (an illusion, but it definitley simplies things for the programs).

Key Abstraction: **The Process**

A **process** is a running program.

A process has:

- a set of CPU registers
  - general purpose registers for the program to use
  - specific registers that contain specific information about this process
    - **instruction pointer** aka **program counter**
    - memory address registers
- its own private memory address space

<img src="images/process_address_space.png" width="150">

This lecture will focus on the **mechanisms** of CPU virtualization. In future lectures, we will discuss **policies**.

How to virtualize the CPU:

To start simply, whenever we want to run a program, load it onto the CPU and just simply run it!

This is **Direct Execution**.

<img src="images/lde.png" width="500">

Three problems with the above simple scheme:

1) What if the process wants to perform a restricted operation?
2) What if the OS wants to switch processes, stop P_a, start P_b.
3) What if a process does something slow, like IO?

# Problem 1: Restricted Ops

The CPU has two modes: Kernel (priviledged) and User (unpriviledged) modes.

We need a way to switch from User and Kernel Mode and back again.

Anytime a user process makes a **System Call**, it is asking the OS to perform some priviledged operation.

Ultimately, sys calls are just functions implemented within the OS, but in order for the CPU to be able to execute them, the CPU Mode must be switched to kernel mode.

When a system call is made, a **trap** is executed. The process traps down to the OS.

The trap:
1) it switched the mode of the CPU to kernel mode
2) runs a trap handler for the system call.

The trap handler looks up in the trap table where the code is for the actual system call and then the OS executes it.

When OS boots, the trap table is built. The trap table contains a mapping from every system call to the actual code in the OS for that system call.

Once the System Call is complete, then the OS returns from the trap back to the user process.

<img src="images/lde_traps.png" width="500">

## Problem #2: Switching Processes

We switch from one process to another by performing a **context switch**. 

A context switch swaps saves the registers of the running process into the kernel stack and loads the registers of the to-be-running process from the kernel onto the cpu.

<img src="images/context_switch.png" width="500">

Since we can switch processes on and off of the CPU, processes can have different states:
1) running
2) ready
3) blocked

## Problem 3: Slow Ops

In order to perform I/O, like writing to a disk, the OS, sends a reqest to the hard disk which will (eventually) execute it, and then let the OS know that it has finished.

Whenever some process makes an IO request, rather than sitting on the CPU and doing nothing while it waits for that request to complete, the OS will switch another process on to make use of the CPU.

A process that is waiting on an IO request to complete (or some other condition to be true) is put in the **blocked state**.

# Limited Direct Execution

The OS directly executes user processes with limits. At key points, the OS gets in the way of user processes to retain control of the whole system.